# Reseau de neuronne

## Importation des librairies

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

## I. Lecture des descripteurs

In [13]:
data = pd.read_csv('../../inference/dataset.csv')
data = np.array(data.values)
label_names = data[:,1024]
unique_label = np.unique(label_names)
feature_values = data[:, :1024]

## II. Réseau de neuronne

On commence par séparer les données en deux ensembles: un pour l'apprentissage et l'autre pour l'évaluation du modèle

In [14]:
# split données
X_train, X_test, y_train, y_test = train_test_split(feature_values, label_names, test_size=0.33, random_state=43)

On poursuit en transformant les labels en indice:

In [15]:
# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(unique_label)
labelIndices_unique = encName2Ind.transform(unique_label)
labelIndices  = encName2Ind.transform(label_names)

# Conversion des noms des labels en indices
labelInd_train = encName2Ind.transform(y_train)
labelInd_test = encName2Ind.transform(y_test)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(labelInd_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(labelInd_test, dtype=tf.float32)

On normalise les données pour réduire la complexité et donner le même poids à tous les descripteurs.

In [16]:
# Normalisation 
images_mean = np.mean(X_train, axis=0, keepdims=True)
images_std = np.std(X_test, axis=0, keepdims=True) + 1e-7
X_train = (X_train - images_mean) / images_std
X_test = (X_test - images_mean) / images_std

/home/nicolas/Documents/3A_ENSTA/ml_embedded/vultr-tensorflow/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


On crée ensuite le réseau de neuronne.

In [17]:
## creation du reseau de neuronne
model = Sequential(name="Sequential_CNN")
# couche de classification
model.add(Dense(256, activation='relu', input_shape = [1024]))
model.add(Dense(len(unique_label), activation='softmax'))
model.summary()

Model: "Sequential_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 264,970
Trainable params: 264,970
Non-trainable params: 0
_________________________________________________________________


Apprentissage :

In [18]:
# paramètres
NB_EPOCHS = 40
LEARNING_RATE = 0.1
MOMENTUM = 0.09
DECAY = 0.01

#  Choix de la tâche à effectuer = choix de la fonction de loss/pertes/coûts à minimise
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy()# sans alias

# compilation ...
model.compile(loss=loss_fcn, optimizer='ADAM', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_asbolute_error', 'mean_squared_error'])

#... et entrainement
history = model.fit(X_train, y_train, epochs=NB_EPOCHS)

Epoch 1/40
21/21 [==============================] - 1s 5ms/step - loss: 1.9894 - accuracy: 0.2960
Epoch 2/40
21/21 [==============================] - 0s 5ms/step - loss: 1.5574 - accuracy: 0.4679
Epoch 3/40
21/21 [==============================] - 0s 5ms/step - loss: 1.3654 - accuracy: 0.5456
Epoch 4/40
21/21 [==============================] - 0s 5ms/step - loss: 1.2385 - accuracy: 0.5815
Epoch 5/40
21/21 [==============================] - 0s 5ms/step - loss: 1.1410 - accuracy: 0.6188
Epoch 6/40
21/21 [==============================] - 0s 5ms/step - loss: 1.0699 - accuracy: 0.6547
Epoch 7/40
21/21 [==============================] - 0s 6ms/step - loss: 0.9927 - accuracy: 0.6846
Epoch 8/40
21/21 [==============================] - 0s 5ms/step - loss: 0.9360 - accuracy: 0.7175
Epoch 9/40
21/21 [==============================] - 0s 5ms/step - loss: 0.8765 - accuracy: 0.7309
Epoch 10/40
21/21 [==============================] - 0s 6ms/step - loss: 0.8600 - accuracy: 0.7160
Epoch 11/40
21/21 [

## III. Evaluation du modèle

In [19]:
#Confution Matrix and Classification Report
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print()
print()
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print()
print()
print('Classification Report')
print(classification_report(labelInd_test, y_pred, target_names=unique_label))

11/11 [==============================] - 0s 2ms/step


Confusion Matrix
[[15  0  3  2  1  0  1  0  1  7]
 [ 0 26  1  0  0  3  0  0  0  0]
 [ 1  0 19  4  4  1  0  0  1  7]
 [ 2  1  1 22  1  1  5  2  2  5]
 [ 0  1  2  2 26  0  1  3  0  1]
 [ 0  9  0  1  0 15  3  0  0  2]
 [ 1  0  0  1  1  0 27  0  0  2]
 [ 0  1  0  2  0  1  0 28  0  0]
 [ 2  1  5  4  5  2  2  2 11  0]
 [ 2  1  2  3  0  2  5  0  2 10]]


Classification Report
              precision    recall  f1-score   support

       blues       0.65      0.50      0.57        30
   classical       0.65      0.87      0.74        30
     country       0.58      0.51      0.54        37
       disco       0.54      0.52      0.53        42
      hiphop       0.68      0.72      0.70        36
        jazz       0.60      0.50      0.55        30
       metal       0.61      0.84      0.71        32
         pop       0.80      0.88      0.84        32
      reggae       0.65      0.32      0.43        34
        rock       0.29      0.3

## IV. Enregistrement de l'ensemble d'évaluation

In [20]:
txt = ""
txt += "#ifndef TEST_DATASET_H\n#define TEST_DATASET_H\n\n"
txt += "#define N_TEST_EXAMPLES " + str(len(X_test)) + "\n"
txt += "#define N_FEATURES " + str(len(y_test)) + "\n"
txt += "#define N_CLASSES 10\n\n"

txt += "float test_acc=" + str(model.evaluate(X_test, y_test)[1]) +";\n\n"

txt += "float X_test[N_TEST_EXAMPLES][N_FEATURES] = {\n"
for i in range(len(X_test)-1):
    txt += "{"
    for j in range(len(y_test)-1):
        txt += str(X_test[i,j].numpy()) +","
    txt += str(X_test[i,len(y_test)-1].numpy()) + "},\n"
txt += "{"
for j in range(len(y_test)-1):
    txt += str(X_test[len(X_test)-1,j].numpy()) +","
txt += str(X_test[len(X_test)-1,len(y_test)-1].numpy()) + "}};\n\n"

txt += "float y_test[N_FEATURES]={"
for i in range(len(y_test)-1):
    txt += str(y_test[i].numpy()) +","
txt += str(y_test[len(y_test)-1].numpy()) +"};\n\n"

txt += "#endif"

# écriture du code dans le fichier
fichier = open("../../inference/NeuronalNetwork/test_dataset.h", "w")
fichier.write(txt)
fichier.close()

11/11 [==============================] - 0s 4ms/step - loss: 1.9973 - accuracy: 0.6030


## V. Enregistrement du modèle sous tensorflow-lite

In [21]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('../../inference/NeuronalNetwork/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp4ucc15yk/assets


INFO:tensorflow:Assets written to: /tmp/tmp4ucc15yk/assets
2023-01-19 00:22:14.147555: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-19 00:22:14.147592: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-19 00:22:14.148082: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp4ucc15yk
2023-01-19 00:22:14.148927: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-19 00:22:14.148943: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp4ucc15yk
2023-01-19 00:22:14.151947: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-19 00:22:14.185824: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp4ucc15yk
2023-01-19 00:22:14.193367: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

## VI. Enregistrement optimisé du modèle sous tensorflowLite

In [22]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open('../../inference/NeuronalNetwork/modelOptimize.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpqszgyv2o/assets


INFO:tensorflow:Assets written to: /tmp/tmpqszgyv2o/assets
2023-01-19 00:22:15.133898: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-19 00:22:15.133976: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-19 00:22:15.134235: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpqszgyv2o
2023-01-19 00:22:15.135523: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-19 00:22:15.135561: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpqszgyv2o
2023-01-19 00:22:15.139798: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-19 00:22:15.189616: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpqszgyv2o
2023-01-19 00:22:15.201394: I tensorflow/cc/saved_model/loader.cc:305] SavedModel